In [1]:
import numpy as np 
import scipy.stats as st
from scipy.stats import shapiro
from scipy.stats import norm
from scipy.stats import f
import statsmodels.api as sm

### 1) Médias de duas Populações independentes - com $σ^2$  conhecidos

Uma transportadora de mercadorias tem duas possibilidades de trajeto para realizar entregas. O gerente de logística desconfia não haver diferença significativa entre o tempo de cada trajeto. Foram selecionadas aleatoriamente 45 entregas realizadas no primeiro trajeto, resultando em uma média amostral de 57 minutos. No segundo trajeto, foram selecionadas aleatoriamente 30 entregas, e o tempo médio foi de 54 minutos. O desvio-padrão populacional do primeiro trajeto é de $σ^2_1$ = 8 minutos, e o do segundo trajeto é de $σ^2_2$ = 6 minutos. Teste a hipótese de que não existe diferença significativa entre o tempo médio dos dois trajetos, ao nível de 1% de significância.

In [2]:
Amostra_trajeto1 = [50.86, 63.98, 58.47, 74.52, 50.53, 50.28, 52.2, 40.01, 52.79, 50.93, 58.2, 59.73, 72.01, 
                    64.6, 52.38, 49.81, 60.94, 46.44, 71.65, 66.44, 53.25, 43.29, 67.83, 56.08, 66.9, 44.24,
                    52.2, 57.04, 57.38, 53.4, 61.98, 48.46, 55.86, 57.96, 61.12, 62.69, 48.0, 44.73, 67.22, 
                    59.66, 51.01, 69.41, 57.93, 66.43, 57.54]

Amostra_trajeto2 = [66.36, 64.53, 52.51, 59.83, 57.87, 62.21, 48.21, 58.12, 60.35, 43.45, 46.9, 41.76,
                   52.38, 58.31, 63.01, 54.44, 63.77, 45.72, 43.78, 53.67, 56.3, 53.8, 41.6, 53.47, 
                   46.17, 58.02, 56.2, 48.36, 50.92, 47.64]


n1, n2 = len(Amostra_trajeto1), len(Amostra_trajeto2)

In [3]:
alpha = 0.05

stat_test, p_valor = shapiro(Amostra_trajeto1) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 1 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal')
    
stat_test, p_valor = shapiro(Amostra_trajeto2) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 2 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal
Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal


In [4]:
def two_sample_ztest(sample1, sample2, sigma1_squared, sigma2_squared, alpha=0.05, alternative="two-sided"):
    """
    Realiza um teste de hipóteses para as médias de duas populações independentes com variâncias conhecidas.

    Argumentos:
    - sample1, sample2: As amostras das duas populações.
    - sigma1_squared, sigma2_squared: As variâncias conhecidas das populações.
    - alpha: Nível de significância (padrão: 0.05).
    - alternative: Especifica a hipótese alternativa. Pode ser "two-sided" (padrão), "less" (menor) ou "greater" (maior).

    Retorna:
    - estatistica_teste: A estatística de teste.
    - p_valor: O valor p do teste de hipóteses.
    - rejeitar_nula: Um booleano indicando se a hipótese nula é rejeitada.
    - intervalo_confianca: O intervalo de confiança para a diferença das médias.
    - valor_critico: O valor crítico para o teste de hipótese com base na hipótese alternativa.
    """
    # Tamanho das amostras
    n1, n2 = len(sample1), len(sample2)

    # Médias das amostras
    mean1, mean2 = np.mean(sample1).round(0), np.mean(sample2).round(0)

    # Diferença das médias
    mean_difference = mean1 - mean2

    # Desvio padrão combinado
    combined_std = np.sqrt((sigma1_squared / n1) + (sigma2_squared / n2)).round(6)

    # Estatística de teste
    z_statistic = mean_difference / combined_std

    # Teste de hipóteses
    if alternative == "two-sided":
        p_value = 2 * (1 - norm.cdf(np.abs(z_statistic)))
        valor_critico = norm.ppf(1 - alpha / 2)
    elif alternative == "less":
        p_value = norm.cdf(z_statistic)
        valor_critico = norm.ppf(alpha)
    elif alternative == "greater":
        p_value = 1 - norm.cdf(z_statistic)
        valor_critico = norm.ppf(1 - alpha)
    else:
        raise ValueError("O argumento 'alternative' deve ser 'two-sided', 'less' ou 'greater'.")

    # Prints solicitados
    print("O tamanho da amostra 1 é de:", n1)
    print("O tamanho da amostra 2 é de:", n2)
    print("A variância da população  1 é:", sigma1_squared)
    print("A variância da população 2 é:", sigma2_squared)
    print("Valor Crítico:", valor_critico.round(3))

    # Testar a hipótese nula
    print("Estatística de teste:", z_statistic.round(3))
    print("P-valor:", p_value.round(3))
    if(p_value <  alpha):
        print("Rejeitamos a Hipótese Nula")
    else:
        print("Não rejeitamos a Hipótese Nula")

    return z_statistic, p_value, valor_critico

In [5]:
sigma1 = 8**2
sigma2 = 6**2

resultado_teste = two_sample_ztest(Amostra_trajeto1, Amostra_trajeto2, sigma1, sigma2, alpha=0.01, alternative="two-sided")

O tamanho da amostra 1 é de: 45
O tamanho da amostra 2 é de: 30
A variância da população  1 é: 64
A variância da população 2 é: 36
Valor Crítico: 2.576
Estatística de teste: 1.853
P-valor: 0.064
Não rejeitamos a Hipótese Nula


### 2) Médias de duas Populações independentes - com $σ^2$  desconhecidos

#### a) Variancias iguais $σ_1^2 = σ_2^2$

Em uma avaliação de estatística, foi selecionada uma amostra de 12 alunos da turma A, resultando em uma média de 7.9 com desvio-padrão 0.6. Na turma B, foram selecionados 15 alunos, os quais tiraram nota média 6.7 com desvio-padrão 0.8. Foram feitos testes de normalidade e igualdade das variâncias e determinou se que as notas possuem distribuição normal e $σ_1^2 = σ_2^2$. Teste a hipótese de que a turma A tem média maior do que a turma B, com um nível de significância de 1%.

In [6]:
TurmaA = [8.04, 7.76, 7.8, 8.46, 8.82, 7.88, 7.4, 7.05, 8.63, 8.25, 8.01, 6.94]
TurmaB = [7.94, 6.07, 6.44, 7.35, 5.72, 6.88, 7.75, 5.41, 6.85, 6.91, 7.33, 5.71, 5.64, 7.12, 6.94]

#### Teste de normalidade  

In [7]:
stat_test, p_valor = shapiro(TurmaA) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 1 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal')
    
stat_test, p_valor = shapiro(TurmaB) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 2 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal
Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal


#### Teste de igualdade/homogeneidade das variâncias  

In [8]:
stat_test, p_valor = st.bartlett(TurmaA, TurmaB)

# Exibir os resultados de forma mais legível
print(f"Teste de Bartlett - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")

if p_valor < alpha:
    print("Rejeita-se a hipótese nula. As variâncias não são iguais.")
else:
    print("Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.")

Teste de Bartlett - Estatística de Teste: 1.12, P-valor: 0.2906
Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.


#### Teste de hipotese independente - Vâriancias iguais $σ_1^2 = σ_2^2$

In [9]:
nivel_significancia = 0.01

# Realizar o teste t de Student para médias independentes
stat_test, p_valor = st.ttest_ind(a=TurmaA, b=TurmaB, equal_var=True, alternative='greater')

print(f"Teste t de Student - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")

if p_valor < nivel_significancia:
    print("Rejeita-se a hipótese nula. A média da Turma A é significativamente maior que a média da Turma B.")
else:
    print("Não há evidências para rejeitar a hipótese nula. Não podemos afirmar que a média da Turma A é maior que a média da Turma B.")

Teste t de Student - Estatística de Teste: 4.52, P-valor: 0.0001
Rejeita-se a hipótese nula. A média da Turma A é significativamente maior que a média da Turma B.


#### b) Vâriancias diferentes $σ_1^2 \neq σ_2^2$

Uma pesquisa avaliou a eficácia de dois tipos de treinamento, com a finalidade de reduzir o tempo médio de determinada tarefa doméstica. Foram selecionadas duas amostras aleatórias e determinou-se através de testes de hipóteses que as populações possuem distribuição normal e $σ_1^2 \neq σ_2^2$. Os dados da pesquisa estão abaixo. Teste a hipótese de que o tempo médio para a realização da tarefa é igual para os dois treinamentos, ao nível de 5% de significância.

In [10]:
Treinamento1 = [19.7, 23.36, 22.84, 26.06, 19.15, 25.66, 30.6, 19.89, 24.8, 22.11, 25.55, 24.26, 22.17, 25.74, 29.9]
Treinamento2 = [16.06, 25.63, 36.46, 14.7, 23.74, 27.1, 24.52, 23.33, 17.4, 25.06]

#### Teste de normalidade

In [11]:
stat_test, p_valor = shapiro(Treinamento1) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 1 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal')
    
stat_test, p_valor = shapiro(Treinamento2) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 2 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal
Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal


#### Teste de igualdade/homogeneidade das variâncias

In [12]:
stat_test, p_valor = st.bartlett(Treinamento1, Treinamento2)

# Exibir os resultados de forma mais legível
print(f"Teste de Bartlett - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")

if p_valor < alpha:
    print("Rejeita-se a hipótese nula. As variâncias não são iguais.")
else:
    print("Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.")

Teste de Bartlett - Estatística de Teste: 4.25, P-valor: 0.0393
Rejeita-se a hipótese nula. As variâncias não são iguais.


#### Teste de hipotese independente - Variâncias diferentes  $σ_1^2 \neq σ_2^2$

In [13]:
nivel_significancia = 0.01

stat_test, p_valor = st.ttest_ind(a=Treinamento1, b=Treinamento2, equal_var=False, alternative='two-sided')

# print(f"Região-crítica: {st.t.ppf(1 - nivel_significancia/2, 13).round(2)}")
    
print(f"Teste t de Student - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")

if p_valor < nivel_significancia:
    print("Rejeita-se a hipótese nula. A média de tempo do Treinamento 1 é diferente da média de tempo do Treinamento 2.")
else:
    print("Não há evidências para rejeitar a hipótese nula. Não podemos afirmar que a média de tempo do Treinamento 1 é diferente da média de tempo do Treinamento 2.")

Teste t de Student - Estatística de Teste: 0.33, P-valor: 0.7467
Não há evidências para rejeitar a hipótese nula. Não podemos afirmar que a média de tempo do Treinamento 1 é diferente da média de tempo do Treinamento 2.


### 3) Proporções de duas Populações Independentes

Em um estudo com 2870 motoristas, 1210 afirmaram ter o hábito de mexer no celular com o carro em
movimento. Depois de sancionada uma multa, foi realizado outro estudo com 2200 motoristas, dos
quais 725 afirmaram ter ainda o hábito. Usando um nível de significância de 10%, é possível verificar a alegação de que a proporção de motoristas com hábito de mexer no celular no trânsito diminuiu significativamente após a criação da multa?

p = proporção de motorista que usam o celular com o carro em movimento. 

$$\begin{cases} H_0 = p_1 - p_2  = 0\\H_1 = p_1 - p_2 > 0 \end{cases} $$

In [14]:
# Dados de exemplo
sucessos_A = 1210  # Número de motorista que afirmaram ter o hábito de mexer no celular com o carro em movimento
total_A = 2870      # Número total de observações na amostra Antes da multa

sucessos_B = 725  #  Número de motorista que afirmaram ter o hábito de mexer no celular com o carro em movimento
total_B = 2200      # Número total de observações na amostra depois da multa

In [15]:
nivel_significancia = 0.10

p_barra = (sucessos_A + sucessos_B) / (total_A + total_B)

sucessos = [sucessos_A, sucessos_B]
totais = [total_A, total_B]

z_stat, p_value = sm.stats.proportions_ztest(sucessos, totais, prop_var = p_barra)

if p_value < nivel_significancia:
    print("Rejeita-se a hipótese nula. Há evidências estatísticas de diferença nas proporções.")
else:
    print("Não há evidências para rejeitar a hipótese nula. Não há diferença estatisticamente significativa nas proporções.")
    
print(f"Teste Z - Estatística de Teste: {round(z_stat, 3)}, P-valor: {round(p_value, 4)}")
print(f"Região-crítica: {norm.ppf(1 - nivel_significancia).round(3)}")

Rejeita-se a hipótese nula. Há evidências estatísticas de diferença nas proporções.
Teste Z - Estatística de Teste: 6.687, P-valor: 0.0
Região-crítica: 1.282


### 4) Teste de hipotese Dependente - Média de duas populações dependentes

Em uma fábrica, sete máquinas foram selecionadas aleatoriamente a fim de determinar o efeito da manutenção preventiva na produção. Inicialmente, as máquinas trabalharam por um período na forma habitual, e depois trabalham o mesmo período recebendo manutenções preventivas. O total de trabalho produzido, antes e depois da adoção das manutenções, está na tabela ao lado. Ao nível de 5%, podemos concluir que o trabalho médio produzido é maior depois da adoção das manutenções preventivas? 

A função *ttest_reg* testa a hipótese nula $H_0$ de que a diferença das médias é igual a zero.

$$\begin{cases} H_0 = \text{Trabalho produzido antes da manutenção e após a manutenção = 0 } =>  \mu_d =0\\H_1 = \text{Trabalho produzido antes da manutenção <  após a manutenção} < 0 => \mu_d < 0 \end{cases} $$

In [16]:
dados_antes = np.array([12.10, 12.30, 11.10, 12.80, 14.10, 8.40, 13.30])
dados_depois = np.array([12.50, 16.00, 12.90, 14.00, 12.90, 12.50, 13.50])

diferenca = dados_antes - dados_depois 
diferenca

array([-0.4, -3.7, -1.8, -1.2,  1.2, -4.1, -0.2])

#### Teste de normalidade 

In [17]:
alpha = 0.05 

stat_test, p_valor = shapiro(diferenca) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - A diferenca não possui dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - A diferenca possui dist. Normal')

Não rejeitamos a Hipótese Nula - A diferenca possui dist. Normal


#### Teste de hipotese pop dependente

In [18]:
alpha = 0.05

stat_test, p_valor = st.ttest_rel(dados_antes, dados_depois, alternative='less')

print(f"Teste t de Student - Estatística de Teste: {round(stat_test, 3)}, P-valor: {round(p_valor, 4)}")
    
if p_valor < alpha:
    print("Rejeita-se a hipótese nula.")
else:
    print("Não há evidências para rejeitar a hipótese nula.")

Teste t de Student - Estatística de Teste: -2.015, P-valor: 0.0452
Rejeita-se a hipótese nula.


### 5) Teste para as Variâncias de duas populações

Nos EUA, as moedas de quarto de dólar sofreram alterações no peso depois de 1964. Uma amostra de 40 moedas fabricadas antes de 1964 resultou em um desvio-padrão de 0.087 g.  Uma amostra de 40 moedas fabricadas depois de 1964 resultou em um desvio-padrão de 0.06194 g. Ao se projetar uma máquina de vendas com moedas, deve-se considerar os desvios-padrão antes e depois de 1964. Use o nível de significância de 5% para testar a afirmativa de que os pesos de quarto de dólar antes e depois de 1964 são provenientes de populações com o mesmo desvio-padrão. 

In [19]:
Antes_1964 = [6.31, 6.3, 6.35, 6.16, 6.06, 6.33, 6.19, 6.08, 6.27, 6.33, 
              6.25, 6.22, 6.25, 6.23, 6.14, 6.2, 6.31, 6.29, 6.27, 6.17, 
              6.24, 6.07, 6.28, 6.38, 6.13, 6.21, 6.29, 6.37, 6.22, 6.2, 
              6.09, 6.27, 6.44, 6.28, 6.26, 6.29, 6.26, 6.19, 6.33, 6.28]


Depois_1964 = [6.31, 6.36, 6.21, 6.29, 6.32, 6.26, 6.12, 6.28, 6.31, 6.2, 6.24, 6.19, 6.1, 6.32,
               6.26, 6.36, 6.31, 6.24, 6.25, 6.3, 6.32, 6.28, 6.33, 6.24, 6.2, 6.26, 6.27, 6.19,
               6.18, 6.29, 6.23, 6.25, 6.24, 6.35, 6.34, 6.18, 6.23, 6.32, 6.3, 6.3]

#### Teste de normalidade 

In [20]:
alpha = 0.05 

stat_test, p_valor = shapiro(Antes_1964) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados Antes_1964 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados Antes_1964 possuem dist. Normal')
    
stat_test, p_valor = shapiro(Depois_1964) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados Depois_1964 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados Depois_1964 possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados Antes_1964 possuem dist. Normal
Não rejeitamos a Hipótese Nula - Os dados Depois_1964 possuem dist. Normal


#### Teste de hipotese - duas populações independentes - teste da variância  

In [21]:
alpha = 0.05

variance_group1 = np.var(Antes_1964, ddof=1)
variance_group2 = np.var(Depois_1964, ddof=1)

F_statistic = variance_group1 / variance_group2

df_num = len(Antes_1964) - 1
df_den = len(Depois_1964) - 1

p_value = 2 * (1 - f.cdf(F_statistic, df_num, df_den))
critical_value = f.ppf(1 - alpha / 2, df_num, df_den)

if p_valor < alpha:
    print("Rejeita-se a hipótese nula. As variâncias não são iguais.")
else:
    print("Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.")
    
print(f"F-statistic: {round(F_statistic, 3)}")
print(f"P-value: {round(p_value, 4)}")
print(f"Critical Region: F > {round(critical_value, 2)}")

Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.
F-statistic: 1.968
P-value: 0.0374
Critical Region: F > 1.89
